In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)
!apt-get install unrar

# Change to the directory containing your RAR file
!cp '/content/drive/My Drive/Project unos__.rar' './'

# Unrar the file
!unrar x 'Project unos__.rar'

Mounted at /content/drive
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.

UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from Project unos__.rar

Extracting  test.json                                                      5%  OK 
Extracting  train.json                                                    51% 94%  OK 
Extracting  validation.json                                              100%  OK 
All OK


In [ ]:
import json
# Replace 'your_file.json' with your JSON file's name
with open('train.json', 'r', encoding='utf8') as file:
    train = json.load(file)
# Replace 'your_file.json' with your JSON file's name
with open('validation.json', 'r', encoding='utf8') as file:
    valid = json.load(file)
# Replace 'your_file.json' with your JSON file's name
with open('test.json', 'r', encoding='utf8') as file:
    test = json.load(file)

In [ ]:
def replace_diacritics(text):
    replacements = {'ă': 'a', 'î': 'i', 'â': 'a', 'ș': 's', 'ț': 't'}
    for diacritic, replacement in replacements.items():
        text = text.replace(diacritic, replacement)
    return text
romanian_text = "Exemplu de text cu diacritice: șarpe, măr, pădure."
processed_text = replace_diacritics(romanian_text)
print(processed_text)

Exemplu de text cu diacritice: sarpe, mar, padure.


In [ ]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# stopwords
#romanian_stopwords = stopwords.words('romanian')

stemmer = SnowballStemmer('romanian')

def inlocuire_diacritice(propozitie):
    inlocuire = {'ă': 'a', 'î': 'i', 'â': 'a', 'ș': 's', 'ț': 't'}
    for diacritice, inlocuiri in inlocuire.items():
       propozitie = propozitie.replace(diacritice, inlocuiri)
    return propozitie


def eliminare_structuri_html(propozitie):
    propozitie = re.sub('<.*?>+', '', propozitie)
    return propozitie


def normalizare(propozitie):

    #litere mici
    propozitie = propozitie.lower()

    #eliminare diacritice
    propozitie = inlocuire_diacritice(propozitie)

    #eliminare html
    propozitie = eliminare_structuri_html(propozitie)

    #elimminare semne de punctuatie
    propozitie = re.sub(r'[^\w\s]', '', propozitie)

    #Tokenizare
    tokens = nltk.word_tokenize(propozitie)

    # stemming
    stemmed_tokens = [stemmer.stem(token) for token in tokens]

    # contopirea tokenilor stemmed
    propozitie = ' '.join(stemmed_tokens)

    return propozitie

def preprocesare(data,nolabel = False):
    labels = []
    guid = []
    propozitie1 = []
    propozitie2 = []
    for inregistrare in data:
        propozitie1.append(normalizare(inregistrare["sentence1"]))
        propozitie2.append(normalizare(inregistrare["sentence2"]))

        if nolabel != True:
            labels.append(inregistrare["label"])

        guid.append(inregistrare["guid"])

    return (propozitie1,propozitie2),labels,guid


preprocessed_train = preprocesare(train)
preprocessed_valid = preprocesare(valid)
preprocessed_test = preprocesare(test,nolabel = True)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
# definiea etichetelor pentru train si validare
labels =  preprocessed_train[1]
labels_one_hot = to_categorical(labels, num_classes=4) #exista 4 clase, realizam one hot encoding
labels_valid = preprocessed_valid[1]
labels_valid_one_hot = to_categorical(labels_valid, num_classes=4)

# tokenzirea propozitiilor
tokenizer = Tokenizer()

propozitie1, propozitie2 = preprocessed_train[0]
propozitie1_valid, propozitie2_valid = preprocessed_valid[0]

# fit pe text, ofera un vocabular cu toate cuvintele unice
tokenizer.fit_on_texts(propozitie1 + propozitie2)

#pentru datele din antrenare
secventa1 = tokenizer.texts_to_sequences(propozitie1)
secventa2 = tokenizer.texts_to_sequences(propozitie2)

#pentru datele din validare
secventa1_valid = tokenizer.texts_to_sequences(propozitie1_valid)
secventa2_valid = tokenizer.texts_to_sequences(propozitie2_valid)


#aflarea lungimii maxime pentru padding
lungime_maxima = max(max(len(s) for s in secventa1), max(len(s) for s in secventa2))

# aplicarea padding ului pentru a avea secvente de aceasi lungime
secventa1_padded = pad_sequences(secventa1, maxlen=lungime_maxima , padding='post')
secventa2_padded = pad_sequences(secventa2, maxlen=lungime_maxima , padding='post')

secventa1_padded_valid = pad_sequences(secventa1_valid, maxlen=lungime_maxima , padding='post')
secventa2_padded_valid = pad_sequences(secventa2_valid, maxlen=lungime_maxima , padding='post')

#combinarea propozitiile pentru construirea vocabularului
toate_propozitiile = preprocessed_train[0][0] + preprocessed_train[0][1]
#crearea unui set care va reprezenta vocabularul, fiind un set, cuvintele sunt unice, neavand duplicate
vocabular = set(cuvant for propozitie in toate_propozitiile for cuvant in propozitie.split())
dimensiune_vocabular = len(vocabular)

# Definirea parametrilor modelului
dimensiune_vocabular = len(tokenizer.word_index) + 1
dimensiune_embedding = 100
lungime_maxima = max(max(len(s) for s in secventa1), max(len(s) for s in secventa2)) #calcularea lungimii secventei maxime
numar_clase = 4 #cele 4 clase


NameError: name 'preprocessed_train' is not defined

In [ ]:
def f1_score(y_true, y_pred):

    # Calcularea preciziei si recall ului pentru fiecare clasa
    TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)), axis=0) #prezicerile corecte pe toate clasele
    total_preziceri = K.sum(K.round(K.clip(y_true, 0, 1)), axis=0) #numarul total de preziceri corecte si incorecte
    predictii_adevarate = K.sum(K.round(K.clip(y_pred, 0, 1)), axis=0) #calcularea numarului total de preziceri corecte de catre model

    precision = TP / (predictii_adevarate + K.epsilon())
    recall = TP / (total_preziceri + K.epsilon())

    # Calcularea f1 score pt fiecare clasa
    f1 = 2 * (precision * recall) / (precision + recall + K.epsilon())

    # Calcularea macro F1 score pe toate clasele
    macro_f1 = K.mean(f1)
    return macro_f1

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Concatenate, Flatten, Dropout, Conv1D, MaxPooling1D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

#Crearea arhitecturii modelului
input1 = Input(shape=(lungime_maxima,))
input2 = Input(shape=(lungime_maxima,))

embedding_layer = Embedding(dimensiune_vocabular, dimensiune_embedding, input_length=lungime_maxima)
embedded1 = embedding_layer(input1)
embedded2 = embedding_layer(input2)

concatenated = Concatenate(axis=1)([embedded1, embedded2]) #concatenarea celor 2 straturi embedded

conv1 = Conv1D(64, 2, activation='relu')(concatenated) #cele 2 straturi Conv1d aplica filtre care invata anumite pattern uri
pool1 = MaxPooling1D(2)(conv1) #reduce dimensionalitatea pentru a putea face presupuneri
conv2 = Conv1D(32, 2, activation='relu')(pool1)
flat = Flatten()(conv2) #se realizeaza un singur vector
dense1 = Dense(200, activation='relu')(flat) #un strat full conectat care proceseaza caracteristici extrase din straturile convolutionale
drop1 = Dropout(0.5)(dense1)
output = Dense(numar_clase, activation='softmax')(drop1) #4 neuroni, care reprezinta fiecare clasa cu o probabilitate data de functia softmax

# Create the model
model = Model(inputs=[input1, input2], outputs=output)

# early stopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10, #verifica validation loss si se va opri dupa 5 epoci daca nu sunt progrese
    restore_best_weights=True #se salveaza cele mai bune weights uri
)

# ModelCheckpoint callback
checkpoint = ModelCheckpoint('my_model.keras', save_best_only=True, monitor='val_loss', mode='min')

# Calcularea class weights
class_weights = compute_class_weight('balanced', classes=np.unique(labels), y=labels) #se creaza weights uri pentru fiecare clasa in vederea dataset ului care nu este echilibrat
class_weights_dict = {i : weight for i, weight in enumerate(class_weights)}


#optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

#Compilarea modelului
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy', f1_score])

# Antrenarea
model.fit([secventa1_padded, secventa2_padded], labels_one_hot,
          validation_data=([secventa1_padded_valid, secventa2_padded_valid], labels_valid_one_hot),
          epochs=30,
          batch_size=256,
          class_weight=class_weights_dict,
          callbacks=[early_stopping,checkpoint])

Epoch 1/30
228/228 [==============================] - 34s 140ms/step - loss: 1.3871 - accuracy: 0.2612 - f1_score: 0.0000e+00 - val_loss: 1.3953 - val_accuracy: 0.1353 - val_f1_score: 0.0000e+00
Epoch 2/30
228/228 [==============================] - 26s 115ms/step - loss: 1.3625 - accuracy: 0.3610 - f1_score: 9.3199e-04 - val_loss: 1.3024 - val_accuracy: 0.4227 - val_f1_score: 0.0014
Epoch 3/30
228/228 [==============================] - 24s 104ms/step - loss: 1.2141 - accuracy: 0.4285 - f1_score: 0.1118 - val_loss: 1.0788 - val_accuracy: 0.5011 - val_f1_score: 0.2002
Epoch 4/30
228/228 [==============================] - 19s 85ms/step - loss: 1.0318 - accuracy: 0.5001 - f1_score: 0.2695 - val_loss: 1.1238 - val_accuracy: 0.4861 - val_f1_score: 0.2295
Epoch 5/30
228/228 [==============================] - 20s 88ms/step - loss: 0.8771 - accuracy: 0.5649 - f1_score: 0.3806 - val_loss: 1.0382 - val_accuracy: 0.5260 - val_f1_score: 0.2771
Epoch 6/30
228/228 [==============================] - 2

In [ ]:
from tensorflow.keras.models import load_model

def f1_score(y_true, y_pred):

    # Calcularea preciziei si recall ului pentru fiecare clasa
    TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)), axis=0) #prezicerile corecte pe toate clasele
    total_preziceri = K.sum(K.round(K.clip(y_true, 0, 1)), axis=0) #numarul total de preziceri corecte si incorecte
    predictii_adevarate = K.sum(K.round(K.clip(y_pred, 0, 1)), axis=0) #calcularea numarului total de preziceri corecte de catre model

    precision = TP / (predictii_adevarate + K.epsilon())
    recall = TP / (total_preziceri + K.epsilon())

    # Calcularea f1 score pt fiecare clasa
    f1 = 2 * (precision * recall) / (precision + recall + K.epsilon())

    # Calcularea macro F1 score pe toate clasele
    macro_f1 = K.mean(f1)
    return macro_f1

#realizam acelasi procedeu la fel ca la preprocesarea setului de antrenare si validare, de data aceasta pentru cel de testare
propozitia1_test, propozitia2_test = preprocessed_test[0]

propozitia1_test = tokenizer.texts_to_sequences(propozitia1_test)
propozitia2_test = tokenizer.texts_to_sequences(propozitia2_test)

propozitia1_padded_test = pad_sequences(propozitia1_test, maxlen=lungime_maxima, padding='post')
propozitia2_padded_test = pad_sequences(propozitia2_test, maxlen=lungime_maxima, padding='post')

#preluam cele mai bune weights uri
best_model = load_model('my_model.keras',custom_objects={'f1_score': f1_score})
#realizam predictia
predicted_labels = best_model.predict((propozitia1_padded_test,propozitia2_padded_test))

predicted_labels = np.argmax(predicted_labels, axis=1)
preprocessed_test_array = np.array(preprocessed_test[2]).reshape((-1, 1))
predicted_labels_array = np.array(predicted_labels).reshape((-1, 1))

import pandas as pd
#crearea csv
print(predicted_labels_array.shape)

data = np.hstack([preprocessed_test_array, predicted_labels_array])
df = pd.DataFrame(data, columns=["guid", "label"])

df.to_csv('CNN2.csv',index=False)

94/94 [==============================] - 3s 28ms/step
(3000, 1)
